# Action Recognition @ UCF101  
**Due date: 11:59 pm on Nov. 19, 2019 (Tuesday)**

## Description
---
In this homework, you will be doing action recognition using Recurrent Neural Network (RNN), (Long-Short Term Memory) LSTM in particular. You will be given a dataset called UCF101, which consists of 101 different actions/classes and for each action, there will be 145 samples. We tagged each sample into either training or testing. Each sample is supposed to be a short video, but we sampled 25 frames from each videos to reduce the amount of data. Consequently, a training sample is an image tuple that forms a 3D volume with one dimension encoding *temporal correlation* between frames and a label indicating what action it is.

To tackle this problem, we aim to build a neural network that can not only capture spatial information of each frame but also temporal information between frames. Fortunately, you don't have to do this on your own. RNN — a type of neural network designed to deal with time-series data — is right here for you to use. In particular, you will be using LSTM for this task.

Instead of training an end-to-end neural network from scratch whose computation is prohibitively expensive, we divide this into two steps: feature extraction and modelling. Below are the things you need to implement for this homework:
- **{35 pts} Feature extraction**. Use any of the [pre-trained models](https://pytorch.org/docs/stable/torchvision/models.html) to extract features from each frame. Specifically, we recommend not to use the activations of the last layer as the features tend to be task specific towards the end of the network. 
    **hints**: 
    - A good starting point would be to use a pre-trained VGG16 network, we suggest first fully connected layer `torchvision.models.vgg16` (4096 dim) as features of each video frame. This will result into a 4096x25 matrix for each video. 
    - Normalize your images using `torchvision.transforms` 
    ```
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    prep = transforms.Compose([ transforms.ToTensor(), normalize ])
    prep(img)
    The mean and std. mentioned above is specific to Imagenet data
    
    ```
    More details of image preprocessing in PyTorch can be found at http://pytorch.org/tutorials/beginner/data_loading_tutorial.html
    
- **{35 pts} Modelling**. With the extracted features, build an LSTM network which takes a **dx25** sample as input (where **d** is the dimension of the extracted feature for each frame), and outputs the action label of that sample.
- **{20 pts} Evaluation**. After training your network, you need to evaluate your model with the testing data by computing the prediction accuracy **(5 points)**. The baseline test accuracy for this data is 75%, and **10 points** out of 20 is for achieving test accuracy greater than the baseline. Moreover, you need to compare **(5 points)** the result of your network with that of support vector machine (SVM) (stacking the **dx25** feature matrix to a long vector and train a SVM).
- **{10 pts} Report**. Details regarding the report can be found in the submission section below.

Notice that the size of the raw images is 256x340, whereas your pre-trained model might take **nxn** images as inputs. To solve this problem, instead of resizing the images which unfavorably changes the spatial ratio, we take a better solution: Cropping five **nxn** images, one at the image center and four at the corners and compute the **d**-dim features for each of them, and average these five **d**-dim feature to get a final feature representation for the raw image.
For example, VGG takes 224x224 images as inputs, so we take the five 224x224 croppings of the image, compute 4096-dim VGG features for each of them, and then take the mean of these five 4096-dim vectors to be the representation of the image.

In order to save you computational time, you need to do the classification task only for **the first 25** classes of the whole dataset. The same applies to those who have access to GPUs. **Bonus 10 points for running and reporting on the entire 101 classes.**


## Dataset
Download **dataset** at [UCF101](http://vision.cs.stonybrook.edu/~yangwang/public/UCF101_images.tar)(Image data for each video) and the **annos folder** which has the video labels and the label to class name mapping is included in the assignment folder uploaded. 


UCF101 dataset contains 101 actions and 13,320 videos in total.  

+ `annos/actions.txt`  
  + lists all the actions (`ApplyEyeMakeup`, .., `YoYo`)   
  
+ `annots/videos_labels_subsets.txt`  
  + lists all the videos (`v_000001`, .., `v_013320`)  
  + labels (`1`, .., `101`)  
  + subsets (`1` for train, `2` for test)  

+ `images/`  
  + each folder represents a video
  + the video/folder name to class mapping can be found using `annots/videos_labels_subsets.txt`, for e.g. `v_000001` belongs to class 1 i.e. `ApplyEyeMakeup`
  + each video folder contains 25 frames  



## Some Tutorials
- Good materials for understanding RNN and LSTM
    - http://blog.echen.me
    - http://karpathy.github.io/2015/05/21/rnn-effectiveness/
    - http://colah.github.io/posts/2015-08-Understanding-LSTMs/
- Implementing RNN and LSTM with PyTorch
    - [LSTM with PyTorch](http://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html#sphx-glr-beginner-nlp-sequence-models-tutorial-py)
    - [RNN with PyTorch](http://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
cd '/content/drive/My Drive/Joshi_Parth_112675594_hw5'

/content/drive/My Drive/Joshi_Parth_112675594_hw5


In [0]:
DONT RUN AGAIN
!wget 'http://vision.cs.stonybrook.edu/~yangwang/public/UCF101_images.tar'

--2019-11-17 22:18:02--  http://vision.cs.stonybrook.edu/~yangwang/public/UCF101_images.tar
Resolving vision.cs.stonybrook.edu (vision.cs.stonybrook.edu)... 130.245.4.232
Connecting to vision.cs.stonybrook.edu (vision.cs.stonybrook.edu)|130.245.4.232|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8658247680 (8.1G) [application/x-tar]
Saving to: ‘UCF101_images.tar’

UCF101_images.tar   100%[===================>]   8.06G  14.2MB/s    in 9m 10s  

2019-11-17 22:27:12 (15.0 MB/s) - ‘UCF101_images.tar’ saved [8658247680/8658247680]



In [0]:
DONT RUN AGAIN
!tar -xkf './UCF101_images.tar' 2>/dev/null

In [0]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob
import random 
import scipy.io
import pandas as pd
from skimage import io, transform
import time
import os
from sklearn.svm import LinearSVC

import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, datasets
from torchsummary import summary

from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

In [0]:
# Reading Text File for actions.txt
actions_file = open('annos/actions.txt', 'r')
actions_list =  [a.split() for a in actions_file.readlines()]

In [ ]:
videos_file = open('annos/videos_labels_subsets.txt', 'r')
videos_list =  [a.split() for a in videos_file.readlines()]
training_labels= []
testing_labels= []
for i in range (len(videos_list)):
  if int(videos_list[i][1]) <= 25:
    if videos_list[i][-1]=='1':
      training_labels.append(videos_list[i][:-1])
    elif videos_list[i][-1]=='2':
      testing_labels.append(videos_list[i][:-1])
print("Train")
print(training_labels)
print("Test")
print(testing_labels)

In [ ]:
DONT RUN AGAIN
# Creating train data folder
os.mkdir('TrainData')

for tr in training_labels:
  img, l = tr[0], tr[1]

  if img in os.listdir('./images'):
    print(img)
    shutil.movetree(('./images/'+img), ('./TrainData/'+img))

In [0]:
DONT RUN AGAIN
# Creating test data folder
os.mkdir('TestData')

for tr in testing_labels:
  img, l = tr[0], tr[1]

  if img in os.listdir('./images'):
    print(img)
    shutil.movetree(('./images/'+img), ('./TestData/'+img))

In [0]:
# Train

data_transform = transforms.Compose([
        # transforms.CenterCrop(224),                             
        transforms.FiveCrop(224),
        transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),
        transforms.Lambda(lambda norms: torch.stack([transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])(norm) for norm in norms])),
        # transforms.ToTensor(),
        # transforms.Normalize(mean=[0.485, 0.456, 0.406],
        #                      std=[0.229, 0.224, 0.225])
    ])

img_dataset = datasets.ImageFolder(root='./TrainData',
                                           transform=data_transform)

dataset_loader = torch.utils.data.DataLoader(img_dataset,
                                            #  batch_size=5, shuffle=False)
                                             num_workers=10)


In [0]:
data_transform_test = transforms.Compose([
        # transforms.CenterCrop(224),                             
        transforms.FiveCrop(224),
        transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),
        transforms.Lambda(lambda norms: torch.stack([transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])(norm) for norm in norms])),
        # transforms.ToTensor(),
        # transforms.Normalize(mean=[0.485, 0.456, 0.406],
        #                      std=[0.229, 0.224, 0.225])
    ])
img_dataset_test = datasets.ImageFolder(root='./TestData',
                                           transform=data_transform_test)

dataset_loader_test = torch.utils.data.DataLoader(img_dataset_test,
                                            #  batch_size=5, shuffle=False,
                                             num_workers=5)

---
---
## **Problem 1.** Feature extraction

In [0]:
vgg = torchvision.models.vgg16(pretrained=True)
for param in vgg.parameters():
    param.requires_grad = False

vnum_ftrs = vgg.classifier[6].in_features
vfeatures = list(vgg.classifier.children())[:-6]
vgg.classifier = nn.Sequential(*vfeatures)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:06<00:00, 86.2MB/s]


In [0]:
summary(vgg.cuda(), (3,224,224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

In [ ]:
DONT RUN AGAIN
# Pass train images and save features

i=1
feats=[]
for batch_index, batch_data in enumerate(dataset_loader):
  # print(batch_data[0])
  out = vgg(torch.squeeze(batch_data[0]).cuda())
  avgs = torch.Tensor.cpu(out).detach().mean(0).numpy()
  feats.append(avgs)
  
  if i % 25 ==0:
    scipy.io.savemat('Features.'+str(int(i/25))+'.mat', {'feature': feats})
    feats=[]
  print(i)
  i+=1

In [ ]:
DONT RUN AGAIN
# Pass test images and save features

i=1
feats_test=[]
for batch_index_test, batch_data_test in enumerate(dataset_loader_test):
  # print(batch_data_test[0])
  out_test = vgg(torch.squeeze(batch_data_test[0]).cuda())
  avgs_test = torch.Tensor.cpu(out_test).detach().mean(0).numpy()
  feats_test.append(avgs_test)
  
  if i % 25 ==0:
    scipy.io.savemat('Features_test.'+str(int(i/25))+'.mat', {'feature': feats_test})
    feats_test=[]
  print(i)
  i+=1

***
***
## **Problem 2.** Modelling

In [0]:
# Training labels: Size 2409

train_label_list=[]
for feat_num, lbl in enumerate(training_labels, start = 1):
  # print(feat_num,lbl[1])
  train_label_list.append(int(lbl[1])-1)
print(len(train_label_list))
# train_label_tensor = torch.LongTensor(train_label_list)
# train_label_tensor.shape

2409


In [0]:
# Testing labels: Size 951


test_label_list=[]
for feat_num, lbl in enumerate(testing_labels, start = 1):
  # print(feat_num,lbl[1])
  test_label_list.append(int(lbl[1])-1)
print(len(test_label_list))
# test_label_tensor = torch.LongTensor(test_label_list)
# test_label_tensor.shape

951


In [0]:
# Load all train features in one variable
l = os.listdir('./')
lab_list=[]
for i in l:
  chk = i.split('.')
  # print(chk)
  if chk[0] == 'Features':
    lab_list.append(int(chk[1]))
lab_list.sort()
train_data_list = []
for i in lab_list:
  path = 'Features.'+ str(i) + '.mat'
  # print(path)
  x = scipy.io.loadmat(path)
  train_data_list.append(x['feature'])

In [0]:
# Shape of Train Image Features

train_data_tensor = torch.FloatTensor(train_data_list)
train_data_tensor.shape

torch.Size([2409, 25, 4096])

In [0]:
# Load all test features in one variable
l = os.listdir('./')
test_lab_list=[]
for i in l:
  chk = i.split('.')
  # print(chk)
  if chk[0] == 'Features_test':
    test_lab_list.append(int(chk[1]))
test_lab_list.sort()
test_data_list = []
for i in test_lab_list:
  path = 'Features_test.'+ str(i) + '.mat'
  # print(path)
  x = scipy.io.loadmat(path)
  test_data_list.append(x['feature'])

In [0]:
# Shape of Test Image Features

test_data_tensor = torch.FloatTensor(test_data_list)
test_data_tensor.shape

torch.Size([951, 25, 4096])

* ##### **Print the size of your training and test data**

In [0]:
train_data = []
for i in range(len(train_data_list)):
   train_data.append([train_data_list[i], train_label_list[i]])

trainloader = torch.utils.data.DataLoader(train_data, shuffle=False, batch_size=15)

In [0]:
test_data = []
for i in range(len(test_data_list)):
   test_data.append([test_data_list[i], test_label_list[i]])

testloader = torch.utils.data.DataLoader(test_data, shuffle=False, batch_size=15)

---
---
## **Problem 3.** Evaluation

* ##### **Print the train and test accuracy of your model** 

In [0]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.fc(out[:, -1, :]) 
        return out

In [0]:
input_dim = 4096
hidden_dim = 256
layer_dim = 1
output_dim = 25

model = LSTMModel(input_dim, hidden_dim, layer_dim, output_dim)
criterion = nn.CrossEntropyLoss()
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

In [0]:
# Number of steps to unroll
seq_dim = 25
num_epochs = 50
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(trainloader):
        images = images.view(-1, seq_dim, input_dim).requires_grad_()
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        iter += 1
        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in testloader:
                images = images.view(-1, seq_dim, input_dim)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum()

            accuracy = 100 * correct / total

            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

Iteration: 500. Loss: 0.5507544875144958. Accuracy: 21
Iteration: 1000. Loss: 0.15713702142238617. Accuracy: 31
Iteration: 1500. Loss: 0.24347448348999023. Accuracy: 41
Iteration: 2000. Loss: 0.7987858653068542. Accuracy: 60
Iteration: 2500. Loss: 0.05483626946806908. Accuracy: 63
Iteration: 3000. Loss: 0.12854555249214172. Accuracy: 63
Iteration: 3500. Loss: 0.056200020015239716. Accuracy: 70
Iteration: 4000. Loss: 0.0255113672465086. Accuracy: 71
Iteration: 4500. Loss: 0.13052716851234436. Accuracy: 73
Iteration: 5000. Loss: 0.20283609628677368. Accuracy: 77
Iteration: 5500. Loss: 0.02679126337170601. Accuracy: 77
Iteration: 6000. Loss: 0.016158049926161766. Accuracy: 78
Iteration: 6500. Loss: 0.01678580604493618. Accuracy: 79
Iteration: 7000. Loss: 0.005019568372517824. Accuracy: 79
Iteration: 7500. Loss: 0.03825908154249191. Accuracy: 78
Iteration: 8000. Loss: 0.013941808603703976. Accuracy: 79


In [0]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = self.fc(out[:, -1, :]) 
        return out

In [0]:
input_dim = 4096
hidden_dim = 256
layer_dim = 2
output_dim = 25

model = LSTMModel(input_dim, hidden_dim, layer_dim, output_dim)
criterion = nn.CrossEntropyLoss()
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

In [0]:
# Number of steps to unroll
seq_dim = 25
num_epochs = 20
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(trainloader):
        images = images.view(-1, seq_dim, input_dim).requires_grad_()
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        iter += 1
        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            for images, labels in testloader:
                images = images.view(-1, seq_dim, input_dim)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum()

            accuracy = 100 * correct / total

            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

Iteration: 500. Loss: 0.031018683686852455. Accuracy: 55
Iteration: 1000. Loss: 0.023472515866160393. Accuracy: 58
Iteration: 1500. Loss: 0.010183637030422688. Accuracy: 62
Iteration: 2000. Loss: 0.01843641884624958. Accuracy: 62
Iteration: 2500. Loss: 0.007846254855394363. Accuracy: 62
Iteration: 3000. Loss: 0.0150957852602005. Accuracy: 59


* ##### **Print the train and test and test accuracy of SVM** 

In [0]:
train_data_arr = np.asarray(train_data_list)
train_data2d = np.reshape(train_data_arr, (2409, 25*4096))

test_data_arr = np.asarray(test_data_list)
test_data2d = np.reshape(test_data_arr, (951, 25*4096))

In [0]:
clf = LinearSVC(C=1.0)
clf.fit(train_data2d, train_label_list)
ypreds = clf.predict(test_data2d)
print ("SVM Accuracy: ",np.sum(test_label_list==ypreds)*100.0/len(test_label_list))

SVM Accuracy:  84.85804416403785


## **Problem 4.** Report

## **Bonus**


* ##### **Print the size of your training and test data**

In [0]:
# Don't hardcode the shape of train and test data
print('Shape of training data is :', )
print('Shape of test/validation data is :', )

* ##### **Modelling and evaluation**

In [0]:
#Write your code for modelling and evaluation

## Submission
---
**Runnable source code in ipynb file and a pdf report are required**.

The report should be of 3 to 4 pages describing what you have done and learned in this homework and report performance of your model. If you have tried multiple methods, please compare your results. If you are using any external code, please cite it in your report. Note that this homework is designed to help you explore and get familiar with the techniques. The final grading will be largely based on your prediction accuracy and the different methods you tried (different architectures and parameters).

Please indicate clearly in your report what model you have tried, what techniques you applied to improve the performance and report their accuracies. The report should be concise and include the highlights of your efforts.
The naming convention for report is **Surname_Givenname_SBUID_report*.pdf**

When submitting your .zip file through blackboard, please
-- name your .zip file as **Surname_Givenname_SBUID_hw*.zip**.

This zip file should include:
```
Surname_Givenname_SBUID_hw*
        |---Surname_Givenname_SBUID_hw*.ipynb
        |---Surname_Givenname_SBUID_hw*.pdf
        |---Surname_Givenname_SBUID_report*.pdf
```

For instance, student Michael Jordan should submit a zip file named "Jordan_Michael_111134567_hw5.zip" for homework5 in this structure:
```
Jordan_Michael_111134567_hw5
        |---Jordan_Michael_111134567_hw5.ipynb
        |---Jordan_Michael_111134567_hw5.pdf
        |---Jordan_Michael_111134567_report*.pdf
```

The **Surname_Givenname_SBUID_hw*.pdf** should include a **google shared link**. To generate the **google shared link**, first create a folder named **Surname_Givenname_SBUID_hw*** in your Google Drive with your Stony Brook account. 

Then right click this folder, click ***Get shareable link***, in the People textfield, enter two TA's emails: ***bo.cao.1@stonybrook.edu*** and ***sayontan.ghosh@stonybrook.edu***. Make sure that TAs who have the link **can edit**, ***not just*** **can view**, and also **uncheck** the **Notify people** box.

Colab has a good feature of version control, you should take advantage of this to save your work properly. However, the timestamp of the submission made in blackboard is the only one that we consider for grading. To be more specific, we will only grade the version of your code right before the timestamp of the submission made in blackboard. 

You are encouraged to post and answer questions on Piazza. Based on the amount of email that we have received in past years, there might be dealys in replying to personal emails. Please ask questions on Piazza and send emails only for personal issues.

Be aware that your code will undergo plagiarism check both vertically and horizontally. Please do your own work.